In [ ]:
#!pip install wget==3.2
#!pip install reverse_geocoder
!pip install kaleido

     |████████████████████████████████| 74.6 MB 84.1 MB/s 
You should consider upgrading via the '/opt/venv/bin/python -m pip install --upgrade pip' command.


In [ ]:
#import required packages
import pandas as pd
import numpy as np
import requests
import wget
import os
import plotly
import plotly.graph_objects as go
import plotly.express as px
from IPython.display import Image
from urllib.request import urlopen
import json
import reverse_geocoder as rg

In [ ]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', -1)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  


## Datasets

### 1. Census
Two datasets from the ACS 1-year 2019 estimates:

-  **DP05** - Demographic and Housing Estimates (exported from the DP05 table at county level summary from [data.census.gov](data.census.gov)).
-  **DP03** - Economic Characteristics (exported from the DP03 table at county level summary from [data.census.gov](data.census.gov)).


### 2. Climate (temperature and precipitation)
The daily Global Historical Climatology Network (GHCN) dataset contains global daily climate data (including temperature and precipitation) across 100,000 stations (read more here: https://www.ncdc.noaa.gov/ghcn-daily-description).


### 3. COVID-19 
The Johns Hopkins University Center for Systems Science and Engineering COVID-19 dataset contains global daily COVID-19 metrics (including daily case rate) at the County level for US data (read more here: https://github.com/CSSEGISandData/COVID-19).


### 4. UID Lookup Table
A key for US County names with their respective FIPS codes also found in the Johns Hopkins University Center for Systems Science and Engineering [github repo](https://github.com/CSSEGISandData/COVID-19). 

In [ ]:
def censusApiToDf(year_estimate = "acs5", features = False, metadata_only = False, geography = "county:*&in=state:*"):
    """
    Takes parameters to extract data for the ACS 2019 DP03 and DP05 datasets.
    Returns a dataframe of ACS features or variables at the county level.
    
    Allows the following parameters to personalize query:
    
    1. year_estimate - Choose between acs1 and acs5 for 1-year and 5-year estimates.
        1-year estimates has more current, but less data.
        5-year estimates has less current, but more data.
        
    2. features - A list of features to extract from ACS data.
    
    3. metadata_only - If True, returns the metadata of features for a given datatable.
    
    4. geography - Specify US locations to extract. Defaults to extracting all counties.
    """
    
    baseAPI = f"https://api.census.gov/data/2019/acs/{year_estimate}"
    
    
    if metadata_only:
        url = f"{baseAPI}/profile/variables"
        
    elif features:
        features = ",".join(features)
        url = f"{baseAPI}/profile?get=NAME,{features}&for={geography}"
        
    else:
        return print("Error: Must either set variables = True, or pass input features to extract.")

    response = requests.get(url)
    jsonResponse = json.loads(response.text)
    
    raw = pd.DataFrame(data= jsonResponse)
    headers = raw.iloc[0]
    raw.columns = headers
    df = raw.loc[1:,:]
    
    return df

In [ ]:
census_metadata = censusApiToDf(metadata_only = True).loc[4:, ["name", "label"]]

In [ ]:
census_metadata.sample(frac=1, random_state=8).head()

,name,label
149,DP04_0040E,Estimate!!BEDROOMS!!Total housing units!!1 bedroom
544,DP04_0061E,Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available
888,DP05_0073PE,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican
909,DP02_0025E,Estimate!!MARITAL STATUS!!Males 15 years and over
995,DP02_0110PE,"Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Northern America"


The census metadata files both contain a feature name, and a label description delimited by "!!". 

This can be split in order to better visualize what each feature name refers to and select feature names based on each column.

In [ ]:
def splitCensusMetadata(df):
    """
    Takes the raw census metdata dataframe and splits the name column by the "!!" delimiter
    """
    split = df.join(df['label'].str.split("!!", expand=True))
    return split

In [ ]:
splitCensusMetadata(census_metadata).sample(frac=1, random_state=8).head()

,name,label,0,1,2,3,4,5,6
149,DP04_0040E,Estimate!!BEDROOMS!!Total housing units!!1 bedroom,Estimate,BEDROOMS,Total housing units,1 bedroom,None,None,None
544,DP04_0061E,Estimate!!VEHICLES AVAILABLE!!Occupied housing units!!3 or more vehicles available,Estimate,VEHICLES AVAILABLE,Occupied housing units,3 or more vehicles available,None,None,None
888,DP05_0073PE,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)!!Puerto Rican,Percent,HISPANIC OR LATINO AND RACE,Total population,Hispanic or Latino (of any race),Puerto Rican,None,None
909,DP02_0025E,Estimate!!MARITAL STATUS!!Males 15 years and over,Estimate,MARITAL STATUS,Males 15 years and over,None,None,None,None
995,DP02_0110PE,"Percent!!WORLD REGION OF BIRTH OF FOREIGN BORN!!Foreign-born population, excluding population born at sea!!Northern America",Percent,WORLD REGION OF BIRTH OF FOREIGN BORN,"Foreign-born population, excluding population born at sea",Northern America,None,None,None


The first field in the label is the type of measurement being captured, **Percent** or **Estimate**.
- For race/ethinicity characteristics of each county, we want **Percent**.
- For economic characteristics of each county we want **Percent** insured, and covered by health insurance, as well as the **Estimate** of median household income.

Unfortunately, the delimited id fields are of unequal lengths making it difficult to query unique columns of interest across the rows. 
Instead of filtering by expanded columns, it may be simpler to simply filter on string patterns within the single column.

In [ ]:
def findPatterns(df, col, patterns):
    """
    Takes a dataframe, specified columns, and a list of string patterns.
    Returns a dataframe with matches to all strings patterns.
    """

    string_masks = (df[col].str.contains(string, regex = True, case = False) for string in patterns)
    comb_mask = np.vstack(string_masks).all(axis=0)
    final_df = df[comb_mask]

    return final_df

In [ ]:
race_ethnicity_cols = findPatterns(df = census_metadata,
                    col = "label",
                    patterns= ["percent!!", 
                    "total", 
                    "one race|hispanic or latino \(of any race\)$", 
                    "black|white|pacific islander$|asian$|hispanic"])

filtered_ethnicity_cols = race_ethnicity_cols[~race_ethnicity_cols["label"].str.contains("Other Asian|!!Other Pacific Islander")]
DP05_cols = filtered_ethnicity_cols["name"].values
filtered_ethnicity_cols

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


,name,label
360,DP05_0052PE,Percent!!RACE!!Total population!!One race!!Native Hawaiian and Other Pacific Islander
493,DP05_0044PE,Percent!!RACE!!Total population!!One race!!Asian
796,DP05_0037PE,Percent!!RACE!!Total population!!One race!!White
942,DP05_0038PE,Percent!!RACE!!Total population!!One race!!Black or African American
1225,DP05_0071PE,Percent!!HISPANIC OR LATINO AND RACE!!Total population!!Hispanic or Latino (of any race)


Next we need to grab economic metrics including:

1. Median income
2. Insurance coverage percent
3. Poverty percent

In [ ]:
income_pat = ["estimate!!", "income", "median household"]
insurance_pat = ["percent!!", "with health insurance coverage$", "noninstitutionalized population!!"]
poverty_pat = ["percent!!", "below the poverty level", "all families$"]

economic_cols = pd.concat([findPatterns(census_metadata, "label", income_pat),
                          findPatterns(census_metadata, "label", insurance_pat),
                          findPatterns(census_metadata, "label", poverty_pat)])

DP03_cols = economic_cols["name"].values
economic_cols

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  


,name,label
159,DP03_0062E,Estimate!!INCOME AND BENEFITS (IN 2019 INFLATION-ADJUSTED DOLLARS)!!Total households!!Median household income (dollars)
678,DP03_0096PE,Percent!!HEALTH INSURANCE COVERAGE!!Civilian noninstitutionalized population!!With health insurance coverage
675,DP03_0119PE,Percent!!PERCENTAGE OF FAMILIES AND PEOPLE WHOSE INCOME IN THE PAST 12 MONTHS IS BELOW THE POVERTY LEVEL!!All families


With our needed features at hand, we can now use the census API to grab county-level data for our features of interest

In [ ]:
census_features_all = list(DP03_cols) + list(DP05_cols)
census_colnames = {"DP05_0037PE": "white_perc",
                    "DP05_0038PE": "black_perc",
                    "DP05_0039PE": "amer_native_perc",
                    "DP05_0044PE": "asian_perc",
                    "DP05_0052PE": "pacific_perc",
                    "DP05_0071PE": "hispanic_perc",
                    "DP03_0062E": "income_med_dollars",
                    "DP03_0096PE": "insurance_perc",
                    "DP03_0119PE": "poverty_perc"}

census_fips = censusApiToDf(year_estimate="acs1", features= census_features_all).rename(columns=census_colnames)
census_fips["FIPS"] = census_fips["state"] + census_fips["county"]
census_fips.drop(columns = ["state", "county"], inplace= True)

census_fips

,NAME,income_med_dollars,insurance_perc,poverty_perc,pacific_perc,asian_perc,white_perc,black_perc,hispanic_perc,FIPS
1,"Jefferson County, Kentucky",59049,94.1,9.8,0.1,3.0,71.3,22.4,5.9,21111
2,"Hennepin County, Minnesota",82369,95.2,5.6,0.1,7.3,71.4,13.1,7.0,27053
3,"Olmsted County, Minnesota",80096,94.7,2.7,0.3,6.3,83.4,6.2,5.2,27109
4,"Scott County, Minnesota",108761,95.4,2.4,0.0,5.2,83.0,4.9,5.3,27139
5,"Faulkner County, Arkansas",57642,90.6,11.4,0.0,1.1,82.6,12.2,4.2,05045
6,"Benton County, Arkansas",69130,90.1,7.7,0.7,3.9,87.2,1.6,17.1,05007
7,"Sedgwick County, Kansas",59716,89.3,7.7,0.0,4.3,77.8,8.6,15.0,20173
8,"Douglas County, Kansas",64233,93.5,6.2,0.0,5.7,82.5,3.5,6.5,20045
9,"Cumberland County, Pennsylvania",75391,93.7,4.5,0.0,4.2,86.9,4.5,4.3,42041
10,"Allegheny County, Pennsylvania",64871,96.3,6.6,0.0,3.9,79.4,12.9,2.3,42003


Next, the weather station dataset needs to be cleaned up.

In [ ]:
!wc -l Data/Raw/daily_global_weather_2020.csv | awk '{print $1" rows in the GHCN dataset"}'



1064284 rows in the GHCN dataset


In [ ]:
!head Data/Raw/daily_global_weather_2020.csv | cat

,Station,Date,TAVG,Latitude,Longitude,Elevation,PRCP
0,AE000041196,2020-01-01,211.0,25.333000000000002,55.516999999999996,34.0,0.0
1,AEM00041194,2020-01-01,217.0,25.255,55.364,10.4,0.0
2,AFM00040938,2020-01-01,54.0,34.21,62.228,977.2,23.0
3,AG000060611,2020-01-01,71.0,28.05,9.6331,561.0,10.0
4,AGE00147708,2020-01-01,99.0,36.72,4.05,222.0,0.0
5,AGE00147716,2020-01-01,119.0,35.1,-1.85,83.0,0.0
6,AGE00147719,2020-01-01,61.0,33.7997,2.89,767.0,0.0
7,AGM00060351,2020-01-01,94.0,36.795,5.874,11.0,0.0
8,AGM00060355,2020-01-01,117.0,36.933,6.95,7.0,0.0


Given the large size of the dataset (more than 1 million rows), rather than read in the entire dataset as a dataframe, we can aggregate the yearly mean temperature, mean preciptation, and temperature variance, and load that into memory.

In [ ]:
def weatherLoadAgg(path):
    """
    Takes a path to the daily global weather csv file.
    Returns a dataframe that aggregates mean temp/precip, and temp variance for each weather station.
    """
    df = pd.read_csv(path)

    # Convert temperature to kelvin to avoid negative values during calculations
    # Note, the temperature in the GHCN dataset is celsius*10
    df["TAVG"] = df["TAVG"]/10 + 273

    # Get mean temp/precip and temp variance for 
    agg_df = df.loc[:,["Station","Latitude", "Longitude", "TAVG","PRCP"]].groupby(["Station", "Latitude", "Longitude"]).agg(
        temp_mean = ("TAVG", np.mean),
        temp_range = ("TAVG", lambda x: (max(x) - min(x))),
        temp_std = ("TAVG", np.std),
        precip_mean = ("PRCP", np.mean)).reset_index()

    #Convert temperture mean to celsius for intepretability
    agg_df["temp_mean"] = agg_df["temp_mean"] - 273
    return agg_df

In [ ]:
weatherpath = "Data/Raw/daily_global_weather_2020.csv"
weather_station_agg = weatherLoadAgg(weatherpath)



In [ ]:
weather_station_agg.describe()

,Latitude,Longitude,temp_mean,temp_range,temp_std,precip_mean
count,4970.000000,4970.000000,4970.000000,4970.000000,4912.000000,4970.000000
mean,35.014808,-12.482464,12.049027,31.239195,7.465532,38.155738
std,25.394591,92.663319,9.329349,16.010358,3.828127,57.425432
min,-78.450000,-179.983000,-57.218545,0.000000,0.070711,0.000000
25%,29.131750,-106.937500,5.463065,20.525000,4.734273,13.125668
50%,42.807650,-2.905300,10.274762,31.200000,7.559488,22.154735
75%,50.133300,68.758075,18.673013,41.000000,9.613719,45.542517
max,82.500000,179.217000,40.100000,82.400000,23.032368,1798.000000


Our aggregated dataset contains 4970 rows which represents each unique weather station. 
The weather stations currently have only latitude and longitude. 
In order to join the weather dataset to the census and covid datasets, the FIPS codes. There are two steps needed to transforms these lat/long values to FIPS codes:

1. Convert latitude and longitude to county/state name using reverse geocoder.
2. Use a lookup table to extract FIPS codes from county/state name. 

In [ ]:
#Reverse geocode latitutude and longitutes to county and state
#Function to reverse_geocode counties

def addCountyStateFromLatLong(dataframe, lat, long, US_only = False):
    """
    Takes a dataframe with latitude and longitude and adds the county and state.
    
    Inputs:
    1) Dataframe 
    2) Latitude Column
    3) Longitude Column
    
    Returns: 
    1) Dataframe with County and State
    """
    
    #Use lat and long as tuples to extract county and state 
    query = rg.search([tuple(x) for x in dataframe[[lat, long]].values])
    
    #Initialize lists to extract county and state from our query
    state = []
    county = []
    country = []
    
    if US_only:
        for i in np.arange(0,len(query)):
            state.append(query[i]["admin1"])
            county.append(query[i]["admin2"])
            country.append(query[i]["cc"])
    else:
        for i in np.arange(0,len(query)):
            state.append(query[i]["admin1"])
            county.append(query[i]["admin2"])
    
    #Create dataframe of filled lists
    if US_only:
        sc_df = pd.DataFrame({"state": state,
                        "county": county,
                        "country": country})
    else:
        sc_df = pd.DataFrame({"state": state,
                        "county": county})
    
    #assert that the query and station dataframes are the same size
    assert len(dataframe) == len(sc_df), "Row lengths don't match for input and output. Check lat/long values in input."
    
    #Merge the Station with county and state horizontally
    if US_only:
        concat_df = pd.concat([dataframe,
                           sc_df],
                           axis = 1)

        concat_df = concat_df[concat_df["country"] == "US"].drop(columns = "country")
    else:
        concat_df = pd.concat([dataframe,
                           sc_df],
                           axis = 1)
    
    return concat_df

In [ ]:
weather_station_state_county = addCountyStateFromLatLong(weather_station_agg, "Latitude", "Longitude", US_only=True)

Loading formatted geocoded file...


In [ ]:
weather_station_state_county.head()

,Station,Latitude,Longitude,temp_mean,temp_range,temp_std,precip_mean,state,county
414,CA001017099,48.7833,-123.1333,10.212757,31.6,5.407322,24.382716,Washington,Whatcom County
415,CA001017101,48.7833,-123.0500,10.361979,15.1,3.856899,22.333333,Washington,Whatcom County
420,CA001018611,48.0333,-123.3333,11.848097,21.0,4.323579,17.923875,Washington,Clallam County
466,CA001054500,54.2500,-133.0500,8.919595,23.3,4.625122,45.777027,Alaska,Annette Island Reserve
467,CA001054503,54.2500,-133.0667,8.667458,24.5,4.549818,45.142373,Alaska,Annette Island Reserve


In [ ]:
def fipsIntToString(sr):
    """
    Takes a series of FIPS integers with NA values
    Returns a FIPS string 5 with 0 padding up front
    """
    string = sr.fillna(0).astype("int32").astype("str")
    padding = string.apply(lambda x: "0"*(5 - len(x)) + x)
    return padding

In [ ]:
#Helper function to join county and state with FIPS from a lookup table
def add_FIPS(dataframe, path):
    """
    This function joins FIPS codes to the state and county outputs of reverse_geocoder
    Inputs:
    1. Dataframe with "state" and "county" columns
    2. Path to FIPS lookup table
    """
    #Download FIPS lookuptable with relevant columns
    lookuptable= pd.read_csv(path).loc[:,["FIPS", "Combined_Key"]]
    
    #Add the FIPS codes
    add_FIPS = dataframe.copy()
    add_FIPS["Combined_Key"] = add_FIPS["county"].str.replace(" County", ", ") + add_FIPS["state"] + ", US"
    climate_FIPS = add_FIPS.merge(lookuptable, on = "Combined_Key")
    climate_FIPS.drop(columns = ["state", "county", "Combined_Key"], inplace = True)

    #Convert FIPS to 5 digit string
    climate_FIPS["FIPS"] = fipsIntToString(climate_FIPS["FIPS"])
    return climate_FIPS

Lastly, we want to aggregate all weather stations within each FIPS code.

In [ ]:
fipspath = "Data/Raw/UID_ISO_FIPS_LookUp_Table.csv"

weather_station_FIPS = add_FIPS(weather_station_state_county, fipspath)
weather_FIPS_agg =  weather_station_FIPS.groupby("FIPS").agg(temp_mean = ("temp_mean", np.mean),
                                                            temp_range = ("temp_range", np.mean),
                                                            temp_std = ("temp_std", np.mean),
                                                            precip_mean = ("precip_mean", np.mean)).reset_index()

In [ ]:
weather_FIPS_agg.head()

,FIPS,temp_mean,temp_range,temp_std,precip_mean
0,01073,19.769444,31.4,6.962110,54.246528
1,01089,18.343390,32.5,7.326767,51.701695
2,01097,21.185085,27.3,5.833545,42.488136
3,01101,21.051042,28.9,6.462203,50.420139
4,01117,19.441667,29.6,6.674477,44.843750


After aggregating temperature features by FIPS codes, we are left with 424 observations.
Each of these represent a county with its

1. Temperature mean
2. Precipitation mean
3. Temperature standard deviation
4. Temperature range

The last dataset that needs to be prepared is the COVID-19 dataset.

In [ ]:
def downloadCOVID(dir="Data/Raw"):
    url = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/12-31-2020.csv"
    
    file = url.split("/")[-1]

    if os.path.exists(dir):
        if os.path.exists(os.path.join(dir, file)):
            print(file, "already exists")
        else:
          wget.download(url=url, out=dir)
    else:
        print(f"The directory {dir} does not exist")
    

In [ ]:
downloadCOVID()

12-31-2020.csv already exists


In [ ]:
def covidUsDf(path = "Data/Raw", file = "12-31-2020.csv"):
    """
    Takes a path and a file containing data from the CSSE COVID-19 Daily Reports

    Returns a dataframe with only US data
    """
    raw = pd.read_csv(os.path.join(path, file))
    us = raw[raw["Country_Region"] == "US"]
    return us

In order to compare each county's COVID-19 infection status adjusted for population, the most useful metric is the **Incident Rate** where:

*Incidence Rate = cases per 100,000 persons*

In [ ]:
covid_US = covidUsDf().loc[:, ["FIPS", "Incident_Rate"]]
covid_US["FIPS"] = fipsIntToString(covid_US["FIPS"])
covid_US.head()

,FIPS,Incident_Rate
649,01001,7499.686767
650,01003,6092.709892
651,01005,6133.030868
652,01007,8189.693668
653,01009,8025.801543


We now have 3 datasets which can be joined by FIPS codes which represent US counties. The datasets have the following features:

1. **Census (2019 estimates)**

    - American Native Proportion of the Population
    - Asian Proportion of the Population
    - Black Proportion of the Population
    - Hispanic Proportion of the Population
    - White Proportion of the Population
    - Income in median dollar (annual per household)
    - Proportion of the Population with Insurance
    - Proportion of the Population below the Poverty Level

<br>

2. **Climate (2020 aggregate from 1070 weather stations)**

    - Temperature mean
    - Temperature variation
    - Precipitation mean

<br> 

3. **COVID-19 (Cumulitive Incidence Rate at the end of 2020, 12-31-2020)**

    - Incidence rate



In [ ]:
census_weather_covid_merge = census_fips.merge(weather_FIPS_agg, how= "inner", on = "FIPS").merge(covid_US, how= "inner", on = "FIPS")
census_weather_covid_merge

,NAME,income_med_dollars,insurance_perc,poverty_perc,pacific_perc,asian_perc,white_perc,black_perc,hispanic_perc,FIPS,temp_mean,temp_range,temp_std,precip_mean,Incident_Rate
0,"Jefferson County, Kentucky",59049,94.1,9.8,0.1,3.0,71.3,22.4,5.9,21111,17.075593,37.200000,8.716195,38.464407,6625.828000
1,"Hennepin County, Minnesota",82369,95.2,5.6,0.1,7.3,71.4,13.1,7.0,27053,11.018305,47.600000,11.705671,23.867797,6815.458157
2,"Olmsted County, Minnesota",80096,94.7,2.7,0.3,6.3,83.4,6.2,5.2,27109,9.478644,50.100000,11.613781,24.437288,5880.866494
3,"Sedgwick County, Kansas",59716,89.3,7.7,0.0,4.3,77.8,8.6,15.0,20173,16.427458,39.400000,9.710894,20.267797,7639.494460
4,"Allegheny County, Pennsylvania",64871,96.3,6.6,0.0,3.9,79.4,12.9,2.3,42003,13.485763,35.400000,9.153268,27.379661,4424.918486
5,"Boone County, Missouri",57013,92.9,8.2,0.1,4.9,81.1,8.1,3.5,29019,15.064407,42.300000,9.575212,34.447458,7400.963078
6,"Harris County, Texas",61618,77.6,12.1,0.1,7.0,61.4,19.1,43.7,48201,23.701356,25.650000,6.033679,35.966102,5043.721789
7,"Dallas County, Texas",61796,77.1,10.7,0.1,6.5,60.0,22.6,40.8,48113,21.564576,31.900000,7.806620,33.655932,7335.110088
8,"McLennan County, Texas",50845,86.9,12.7,0.0,1.4,79.8,13.7,27.0,48309,21.550508,30.900000,7.639905,32.827119,7318.128149
9,"Jefferson County, Texas",55797,79.6,13.0,0.3,4.0,58.7,33.5,22.1,48245,22.560000,22.300000,5.520927,38.240678,5408.542524


According to the United States Census Bureau, there are 3,141 county-equivalents in the U.S., but our merged dataset only contains 228 records. 
Which datasets are lacking the most counties?

In [ ]:
def fipsGetAll():
    """
    Downloads and returns all unique US FIPs codes from kjhealy's github repo as integers.
    """
    df = pd.read_csv("https://raw.githubusercontent.com/kjhealy/fips-codes/master/county_fips_master.csv", 
                     encoding = 'cp1252')
    fips = df.loc[:,["fips"]].rename(columns = {"fips": "FIPS"})
    return fips

In [ ]:
all_fips = fipsGetAll()
all_fips["FIPS"] = fipsIntToString(all_fips["FIPS"])
all_fips

,FIPS
0,01001
1,01003
2,01005
3,01007
4,01009
...,...
3141,56037
3142,56039
3143,56041
3144,56043


In [ ]:
def fipsCoverage(df, allfips):
    """
    Takes a dataframe with some FIPS codes and compares it to another dataframe with all FIPS codes, both of which must have a "FIPS" columns.
    Returns the percent of FIPS codes covered in the first dataframe.
    """
    proportion = int(df.merge(allfips, on = "FIPS").shape[0]/ allfips.shape[0]*100)
    return proportion

In [ ]:
print(f"There are {census_fips.shape[0]} counties in the Census dataset with {fipsCoverage(census_fips, all_fips)}% coverage.")
print(f"There are {weather_FIPS_agg.shape[0]} counties in the Climate dataset with {fipsCoverage(weather_FIPS_agg, all_fips)}% coverage.")
print(f"There are {covid_US.shape[0]} counties in the COVID-19 dataset with {fipsCoverage(covid_US, all_fips)}% coverage.")
print(f"There are {census_weather_covid_merge.shape[0]} counties in the merged dataset with {fipsCoverage(census_weather_covid_merge, all_fips)}% coverage.")

There are 840 counties in the Census dataset with 26% coverage.
There are 424 counties in the Climate dataset with 13% coverage.
There are 3274 counties in the COVID-19 dataset with 99% coverage.
There are 228 counties in the merged dataset with 7% coverage.


In order to visualize which US counties, we can map the FIPS codes from each dataset.

In [ ]:
# Map of U.S. Counties
def mapCounties(df, title = "US Counties", savefig = False, fig_dir ="Figures"):

    """
    Takes a dataframe with a "FIPS" column.
    Returns a map of the US with the present counties highlighted.
    """    

    with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
        counties = json.load(response)

    fig = px.choropleth(df, 
                        geojson=counties, 
                        locations='FIPS',
                        scope="usa")

    fig.update_layout(title = title,
                      showlegend = False)
    if savefig:
        if "html" in savefig:
            return plotly.offline.plot(fig, filename= os.path.join(fig_dir, savefig))
        else:
            return fig.write_image(os.path.join(fig_dir, savefig))
    else:
        return fig

In [ ]:
mapCounties(census_fips, 
title = f'Counties in United States 2019 Census Estimates ({fipsCoverage(census_fips, all_fips)}% coverage)',
savefig= "Counties_2019_ACS1_Census.html")

'Figures/Counties_2019_ACS1_Census.html'

In [ ]:
mapCounties(weather_FIPS_agg, 
title= f"Counties in GHCN weather dataset ({fipsCoverage(weather_FIPS_agg, all_fips)}% coverage)",
savefig= "Counties_2019_ACS1_Weather.html")

'Figures/Counties_2019_ACS1_Weather.html'

In [ ]:
mapCounties(covid_US, 
title= f"Counties in COVID-19 dataset ({fipsCoverage(covid_US, all_fips)}% coverage)",
savefig= "Counties_2019_ACS1_COVID19.html")

'Figures/Counties_2019_ACS1_COVID19.html'

In [ ]:
mapCounties(census_weather_covid_merge, 
title= f"Counties in Merged Census, Weather, and COVID Dataset({fipsCoverage(census_weather_covid_merge, all_fips)}% coverage)", 
savefig= "Counties_2019_ACS1_Census_Weather_COVID.html")

'Figures/Counties_2019_ACS1_Census_Weather_COVID.html'

From the maps we can see that:

1. The **COVID-19** dataset has the most coverage, with data on nearly all counties.
2. The **Census 2019 estimates** dataset has moderate coverage, with sparse coverage in the Western US.
3. The **Weather** dataset has modertely low coverage, with sparse coverage in the Eastern US.

After performing inner joins on all three datasets, the full dataset is very limited.

The final, merged, census, weather, and covid dataset has only has 228 records which represents only 7% of all US counties.

There are several reasons why we have such little coverage of US counties including:

1. **Weather stations are not distrubted evenly among counties**. 
    - They appear to be most prevalent along the Rocky Mountains in the Western US.
    - They are sparsley distributed in the MidWest and Eastern US.
2. **The census data is missing many counties**
    - The 1-year 2019 ACS census estimates miss many counties in order to present only the most current information:
        - Only including 2019 data
        - Only including counties with populations > 65,000
        - To read more about the tradeoff between 1-year, 3-year, and 5-year estimates, visit https://www.census.gov/programs-surveys/acs/guidance/estimates.html

This **smaller dataset with the most current feature estimates seems the most appropriate for inference** regarding the effect of demographic and weather features on COVID-19 infection rates.

**For predictions, we can instead want to use the larger, but less current 5-year ACS census estimates**.
Since the Climate dataset 13% coverage, we will keep weather out of the predictive modeling features leaving only demographic features to predict COVID-19 infection rates.

In [ ]:
# Use the same features extracted for the acs1 census estimates

acs5_census= censusApiToDf(year_estimate="acs5", features= census_features_all).rename(columns=census_colnames)
acs5_census["FIPS"] = acs5_census["state"] + acs5_census["county"]
acs5_census.drop(columns = ["state", "county"], inplace= True)

In [ ]:
acs5_covid_merge = acs5_census.merge(covid_US, how= "inner", on = "FIPS")

In [ ]:
acs5_covid_merge

,NAME,income_med_dollars,insurance_perc,poverty_perc,pacific_perc,asian_perc,white_perc,black_perc,hispanic_perc,FIPS,Incident_Rate
0,"Fayette County, Illinois",46650,91.8,12.8,0.1,0.5,93.9,4.7,1.9,17051,12485.939258
1,"Logan County, Illinois",57308,95.5,5.6,0.0,0.8,88.5,6.9,3.4,17107,9452.093088
2,"Saline County, Illinois",44090,95.8,17.9,0.2,0.7,92.7,2.6,1.8,17165,7509.258865
3,"Lake County, Illinois",89427,93.2,5.8,0.0,7.7,75.8,6.8,21.7,17097,6934.324908
4,"Massac County, Illinois",47481,94.6,13.6,0.0,0.2,91.1,5.8,2.9,17127,6484.170781
...,...,...,...,...,...,...,...,...,...,...,...
3189,"Crockett County, Tennessee",44717,88.9,13.1,0.0,0.3,79.5,13.0,10.6,47033,11531.974701
3190,"Lake County, Tennessee",35191,88.7,25.6,0.0,0.2,67.5,28.5,2.3,47095,20068.415051
3191,"Knox County, Tennessee",57470,92.2,9.5,0.0,2.2,85.5,8.7,4.3,47093,6942.610559
3192,"Benton County, Washington",69023,92.7,8.6,0.1,2.6,82.1,1.6,21.7,53005,5720.436421


In [ ]:
print(f"There are {acs5_census.shape[0]} counties in the ACS 5-year Census dataset with {fipsCoverage(acs5_census, all_fips)}% coverage.")
print(f"There are {acs5_covid_merge.shape[0]} counties in the merged prediction dataset with {fipsCoverage(acs5_census, all_fips)}% coverage.")

There are 3220 counties in the ACS 5-year Census dataset with 99% coverage.
There are 3194 counties in the merged prediction dataset with 99% coverage.


The 5-year estimates contain an incredible 99% coverage! The 3,220 records has sufficient data for training and validation in our predictive modeling.

In [ ]:
def write_csv(df, path, filename):
    """
    Takes a dataframe, path, and filename.
    Returns a written csv.
    Checks if the csv is already written, and if the path is correct.
    """
    file = os.path.join(path, filename)

    if os.path.exists(path):
        if os.path.exists(file):
            print(f"The csv {filename} already exists.")
        else:
            pd.DataFrame.to_csv(df, file)
    else:
        print(f"The path {path} does not exist.")

In [ ]:
census_weather_covid_inference_path = "Data/Clean/"
census_weather_covid_inference_name = "census_weather_covid_inference.csv"
write_csv(census_weather_covid_merge, census_weather_covid_inference_path, census_weather_covid_inference_name)

In [ ]:
census_covid_prediction_path = "Data/Clean/"
census_covid_prediction_name = "census_covid_prediction.csv"
write_csv(acs5_covid_merge, census_covid_prediction_path, census_covid_prediction_name)

In [ ]:
# With our 2 preprocessed datasets, we can now move on to analysis.